# Assignment 2: Iterative-Deepening Search

Nathan Kluth

## Overview

This notebook implements the iterative-deepening search algorithm as discussed in [CS440 Week 2 lecture notes](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/07%20Informed%20Search.ipynb).

## Required Code

Below, I have implemented the following functions:

  * `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
  * `depthLimitedSearch(startState, goalState, actionsF, takeActionF, depthLimit)`
  
`depthLimitedSearch` is called by `iterativeDeepeningSearch` with `depthLimit`s of $0, 1, \ldots, $ `maxDepth`. Both return either the solution path as a list of states, or the strings `cutoff` or `failure`.  `failure` signifies that all states were searched and the goal was not found. `cutoff` means that the `maxDepth` was reached without a solution

Each receives the arguments

  * the starting state, 
  * the goal state,
  * a function `actionsF` that is given a state and returns a list of valid actions from that state,
  * a function `takeActionF` that is given a state and an action and returns the new state that results from applying the action to the state,
  * either a `depthLimit` for `depthLimitedSearch`, or `maxDepth` for `iterativeDeepeningSearch`.

In [1]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    """ takes in a maxDepth and works its way up to that depth, calling into
    depthLimitedSearch along the way"""
    
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            # Add startState to front of solution path, in result, returned by depthLimitedSearch       
            return [startState] + result
    return 'cutoff'

In [2]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    """given a depth, recursively checks the tree to find child states when running the 
    possible actions generated by actionsF through takeActionF"""
    
    if (state == goalState):
        return []
    
    if (depthLimit == 0):
        return 'cutoff' # signal that the depth limit was reached
    
    cutoffOccurred = False
    for action in actionsF(state):
        childState = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)

        if result == 'cutoff':
            cutoffOccurred = True
        
        elif result != 'failure':
            # Add childState to front of partial solution path, in result, returned by depthLimitedSearch
            return [childState] + result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

# 8-puzzle

Here, I use the two functions to solve the 8-puzzle.
The state of the puzzle is implemented as a list of integers. 0 represents the empty position. 

Functions for the 8-puzzle are the following:

  * `findBlank_8p(state)`: return the row and column index for the location of the blank (the 0 value).
  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. Return them in the order `left`, `right`, `up`, `down`, though only if each one is a valid action.
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state`.
  * `printPath_8p(startState, goalState, path)`: print a solution path in a readable form.

In [3]:
def findBlank_8p(state):
    """returns the row and column index of the blank (0 value) as tuple (row, column)"""
    blankIndex = state.index(0)
    if blankIndex > 2 and blankIndex < 6:
        idx1 = 1
        idx2 = blankIndex - 3
        
    elif blankIndex > 5:
        idx1 = 2
        idx2 = blankIndex - 6
        
    else:
        idx1 = 0
        idx2 = blankIndex

    return (idx1, idx2)

In [4]:
def actionsF_8p(state):
    """generates the list of actions available for the blank ie where it can move to"""
    blankState = findBlank_8p(state)
    blank = (blankState[0] * 3) + blankState[1]
    
    actions = []
    if blank > 0:
        actions.append("left")
        
    if blank < 8:
        actions.append("right")
        
    if blankState[0] > 0:
        actions.append("up")

    if blankState[0] < 2:
        actions.append("down")
    
    return actions

In [5]:
import copy

def takeActionF_8p(state, action):
    """ given the state of the board, adjust based on the 
    action given to return a new state"""
    
    blankState = findBlank_8p(state)
    blank = (blankState[0] * 3) + blankState[1]
    
    if (action == 'down'):
        belowBlank = blank + 3
        newState = copy.copy(state)
        
        newState[belowBlank], newState[blank] = newState[blank], newState[belowBlank]
        return newState
        
    if (action == 'up'):
        aboveBlank = blank - 3
        newState = copy.copy(state)
        
        newState[aboveBlank], newState[blank] = newState[blank], newState[aboveBlank]
        return newState
        
    if (action == 'left'):
        lefOfBlank = blank - 1
        newState = copy.copy(state)
        
        newState[lefOfBlank], newState[blank] = newState[blank], newState[lefOfBlank]
        return newState
    
    if (action == 'right'):
        rightOfBlank = blank + 1
        newState = copy.copy(state)
        
        newState[rightOfBlank], newState[blank] = newState[blank], newState[rightOfBlank]
        return newState
        
    # default
    return state

In [6]:
def printPath_8p(startState, goalState, path):
    """calls into printState_8p with the path list generated from running the search
    pretty prints the states of the board along the route to the solution"""
    
    print("Path from")
    printState_8p(startState)
    print('  to')
    printState_8p(goalState)
    print('is', len(path), 'nodes long:')

    for (idx, p) in enumerate(path):
        i = 0 # loop index
        print()
        padding = "".rjust(idx)
        while i < 7:
            print(padding, p[i] or ' ', p[i+1] or ' ', p[i+2] or ' ')
            i += 3        

In [7]:
def printState_8p(state):
    """pretty prints the state of the board instead of a list in the form
    of 
       1 2 3
       4 0 5
       6 7 8
    """
    i = 0 # loop index
    
    while i < 7:
        print(state[i] or ' ', state[i+1] or ' ', state[i+2] or ' ')
        i += 3

Here are some example results.

In [8]:
startState = [1, 0, 3, 4, 2, 5, 6, 7, 8]

In [9]:
printState_8p(startState)  # not a required function for this assignment, but it helps when implementing printPath_8p

1   3
4 2 5
6 7 8


In [10]:
findBlank_8p(startState) # blank is in the 1st row, 2nd position

(0, 1)

In [11]:
actionsF_8p(startState) # start state cannot go 'up' since it is on the top row already

['left', 'right', 'down']

In [12]:
takeActionF_8p(startState, 'down') # move the blank down, when shown in list this means adding 3 to the index of 0

[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [13]:
printState_8p(takeActionF_8p(startState, 'down')) # print the previous state out

1 2 3
4   5
6 7 8


In [14]:
goalState = takeActionF_8p(startState, 'down')

In [15]:
goalState

[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [16]:
newState = takeActionF_8p(startState, 'down')

In [17]:
newState == goalState # states can be compared with '=='

True

In [18]:
startState

[1, 0, 3, 4, 2, 5, 6, 7, 8]

Now that the basic setup is done, I can pass those functions in to `depthLimitedSearch` with a random depth to see what I get!

In [19]:
path = depthLimitedSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

[[0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Notice that `depthLimitedSearch` result is missing the start state.  This is inserted by `iterativeDeepeningSearch`.

But, when I try `iterativeDeepeningSearch` to do the same search, it finds a shorter path!

In [20]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path #shorter path found with iterative

[[1, 0, 3, 4, 2, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8]]

In [21]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path # result wasn't found in the limit of 3

'cutoff'

In [22]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 5)
path # same deal, but this time trying 5 depths

'cutoff'

Humm...maybe we can't reach the goal state from this state.  We need a way to randomly generate a valid start state.

In [23]:
import random

In [24]:
random.choice(['left', 'right'])

'right'

In [25]:
def randomStartState(goalState, actionsF, takeActionF, nSteps):
    state = goalState
    for i in range(nSteps):
        state = takeActionF(state, random.choice(actionsF(state)))
    return state

In [26]:
startState = randomStartState(goalState, actionsF_8p, takeActionF_8p, 10)
startState

[1, 2, 0, 4, 5, 3, 6, 7, 8]

In [27]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 20)
path

[[1, 2, 0, 4, 5, 3, 6, 7, 8],
 [1, 2, 3, 4, 5, 0, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Let's print out the state sequence in a readable form.

In [28]:
for p in path:
    printState_8p(p)
    print()

1 2  
4 5 3
6 7 8

1 2 3
4 5  
6 7 8

1 2 3
4   5
6 7 8



Here is one way to format the search problem and solution in a readable form.

In [29]:
printPath_8p(startState, goalState, path)

Path from
1 2  
4 5 3
6 7 8
  to
1 2 3
4   5
6 7 8
is 3 nodes long:

 1 2  
 4 5 3
 6 7 8

  1 2 3
  4 5  
  6 7 8

   1 2 3
   4   5
   6 7 8


Let's make sure it handles a simple case

In [30]:
startState = [0, 1, 2, 3, 4, 5, 6, 7, 8]
goalState = [0, 1, 2, 3, 4, 5, 6, 7, 8] 
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 20)
printPath_8p(startState, goalState, path)

Path from
  1 2
3 4 5
6 7 8
  to
  1 2
3 4 5
6 7 8
is 1 nodes long:

   1 2
 3 4 5
 6 7 8


What about an impossible case?

In [40]:
startState = [0, 1, 2, 3, 4, 5, 6, 7, 8]
goalState = [0, 1, 2, 3, 4, 5, 6, 7, 9] #where did that 9 come from? 
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 15)
path # never going to find it!

'cutoff'

## Towers of Hanoi

Below, I use the same two functions to solve for the [Towers of Hanoi problem](https://en.wikipedia.org/wiki/Tower_of_Hanoi)

Functions are the following:

* `findBlank_tower(state)`: return the pillars that have a space to move.
* `actionsF_tower(state)`: returns a list of valid actions that can be applied in state.
* `takeActionF_tower`(state, action): return the state that results from applying action in state.
* `printPath_tower`(startState, goalState, path): print a solution path in a readable form.

In [579]:
def actionsF_tower(state):
    """ given the state of the columns, generate possible next
    states and return (column, direction, number), eg a disk can move two columns"""
    
    actions = []
    for (idx, column) in enumerate(state):
        if len(column): # tower has a disk
            disk = column[0]
            if (len(state) > idx + 1): # there is a column to the right
                if (len(state[idx+1]) == 0 or state[idx+1][0] > disk): # column is empty OR has a larger disk
                    actions.append((idx, 'right', 1))
                  
            if (idx > 0): # there is a column to the left
                if (len(state[idx-1]) == 0 or state[idx-1][0] > disk): # column is empty OR has a larger disk
                    # prevent substates that existed already
                    actions.append((idx, 'left', 1))
                    
            if (idx > 1): # there is sa column 2 to the left
                if (len(state[idx-2]) == 0 or state[idx-2][0] > disk): # column is empty OR has larger disk
                    # prevent substates that existed already
                    actions.append((idx, 'left', 2))
                    
            if (len(state) > idx + 2): # there is a column 2 to the right
                if (len(state[idx+2]) == 0 or state[idx+2][0] > disk): #column is empty OR has larger disk
                    actions.append((idx, 'right', 2))
    return actions

In [580]:
import copy

def takeActionF_tower(state, action):
    """given the tuple (column, direction, number) move from column, in the direction
    the number of columns"""
    
    newState = copy.deepcopy(state)
    if (action[1] == 'right'):
        newState[action[0] + action[2]].insert(0, newState[action[0]][0]) # move to new positin
    elif (action[1] == 'left'):
        newState[action[0] - action[2]].insert(0, newState[action[0]][0]) # move to new positin
        
    del newState[action[0]][0] # delete from old position
    return newState

In [581]:
def printPath_tower(state, goalState, path):
    """pretty prints the path of the solution"""
    
    print("Path from")
    print(startState)
    print('to')
    print(goalState)
    print('is', len(path), 'nodes long:')

    for (idx, p) in enumerate(path):
        print()
        padding = " ".rjust(idx)
        print(padding, p)

In [582]:
startState = [[1, 2, 3, 4], [], []] # each nested list represents a column, sizes are small to large
startState

[[1, 2, 3, 4], [], []]

In [583]:
actionsF_tower(startState) # shows that only the top most disk can move to the right one

[(0, 'right', 1), (0, 'right', 2)]

In [584]:
print(takeActionF_tower(startState, (0, 'right', 1))) # move the top disk to the right
startState # remains unchanged

[[2, 3, 4], [1], []]


[[1, 2, 3, 4], [], []]

In [585]:
nextState = [[2, 3, 4, 5], [1], []]
takeActionF_tower(nextState, (0, 'right', 2)) # cannot move 1 to the right

[[3, 4, 5], [1], [2]]

In [586]:
testState = [[5], [1, 3], [2, 4]]
actionsF_tower(testState)

[(1, 'right', 1), (1, 'left', 1), (2, 'left', 2)]

In [587]:
midState = [[2, 3, 4, 5], [1], []]
actionsF_tower(midState) 
# now there will be 3 possible actions, disk 1 can be moved left or right
# or disk 2 all the way to the right

[(0, 'right', 2), (1, 'right', 1), (1, 'left', 1)]

In [588]:
biggerState = [[3, 4, 5], [1], [2]]
actionsF_tower(biggerState) # 2 COULD move left but I leave it out to prevent lots of looping

[(1, 'right', 1), (1, 'left', 1), (2, 'left', 2)]

In [589]:
goalState = [[], [], [1, 2, 3, 4]] # all stacked nice on the right!

In [590]:
path = depthLimitedSearch(startState, goalState, actionsF_tower, takeActionF_tower, 8)
path # too shallow

'cutoff'

In [591]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_tower, takeActionF_tower, 20)
path # trying from 15

[[[1, 2, 3, 4], [], []],
 [[2, 3, 4], [1], []],
 [[3, 4], [1], [2]],
 [[3, 4], [], [1, 2]],
 [[4], [3], [1, 2]],
 [[1, 4], [3], [2]],
 [[1, 4], [2, 3], []],
 [[4], [1, 2, 3], []],
 [[], [1, 2, 3], [4]],
 [[], [2, 3], [1, 4]],
 [[2], [3], [1, 4]],
 [[1, 2], [3], [4]],
 [[1, 2], [], [3, 4]],
 [[2], [1], [3, 4]],
 [[], [1], [2, 3, 4]],
 [[], [], [1, 2, 3, 4]]]

In [592]:
printPath_tower(startState, goalState, path)

Path from
[[1, 2, 3, 4], [], []]
to
[[], [], [1, 2, 3, 4]]
is 16 nodes long:

  [[1, 2, 3, 4], [], []]

  [[2, 3, 4], [1], []]

   [[3, 4], [1], [2]]

    [[3, 4], [], [1, 2]]

     [[4], [3], [1, 2]]

      [[1, 4], [3], [2]]

       [[1, 4], [2, 3], []]

        [[4], [1, 2, 3], []]

         [[], [1, 2, 3], [4]]

          [[], [2, 3], [1, 4]]

           [[2], [3], [1, 4]]

            [[1, 2], [3], [4]]

             [[1, 2], [], [3, 4]]

              [[2], [1], [3, 4]]

               [[], [1], [2, 3, 4]]

                [[], [], [1, 2, 3, 4]]


In [593]:
badState = [[9, 9, 9], [9, 9], [9, 9]]
path = iterativeDeepeningSearch(badState, goalState, actionsF_tower, takeActionF_tower, 15)
path # trying from 15, no solution... fails!

'failure'